In [1]:
import requests
import pandas as pd

from pathlib import Path

In [2]:
!ls ../data/

In [3]:
raw_data_dir = Path("../data/raw")
interim_data_dir = Path("../data/interim")

In [4]:
WNDP_API = "https://www.wrmd.org/api/v2/ml/training-dataset/clinical-classifications"

In [5]:
%%time
r = requests.get(WNDP_API)

CPU times: total: 250 ms
Wall time: 29.7 s


In [6]:
r = r.json()

In [7]:
df = pd.DataFrame(r)
df.head(20)

,patient_id,text,terms
0,1960540,unknown. A9 - orphaned. Feathers / Fur / Skin:...,[Clinically healthy]
1,548645,HBC. HBC/NSF. Eyes / Ears / Mouth / Nares: squ...,[Physical injury]
2,1905573,"Dragging leg, circling. Shock, Bleeding from M...",[Physical injury]
3,2741655,Weak. NSF. Eyes / Ears / Mouth / Nares: Eyes s...,[Clinically healthy]
4,2637768,Removed / Displaced (Juvenile) - Nest / Nest s...,[Nonspecific]
5,533717,can't fly. Eyes / Ears / Mouth / Nares: R eye ...,"[Physical injury, Ocular disease]"
6,1880593,suspect lead or PMV. Neurologic: severe head t...,[Neurologic disease]
7,1920531,Tree Felling. No Injuries found - healthy,[Clinically healthy]
8,1955498,Dog interaction. Clinically healthy,[Clinically healthy]
9,2885403,Orphaned. NSF. NSF,[Clinically healthy]


In [8]:
df.terms.value_counts()[:20]

terms
[Clinically healthy]                           3890
[Physical injury]                              2864
[Nonspecific]                                   948
[Nutritional disease]                           763
[Neurologic disease]                            701
[Neurologic disease, Physical injury]           453
[Gastrointestinal disease]                      423
[Dermatologic disease]                          323
[Physical injury, Neurologic disease]           305
[Hematologic disease]                           219
[Ocular disease]                                187
[Respiratory disease, Physical injury]          161
[Respiratory disease]                           144
[Ocular disease, Physical injury]               119
[Urogenital disease]                            116
[Nutritional disease, Physical injury]          113
[Respiratory disease, Neurologic disease]        97
[Physical injury, Respiratory disease]           92
[Physical injury, Ocular disease]                90
[Gastr

In [9]:
df.shape

(14044, 3)

In [11]:
df.to_csv(raw_data_dir/"wndp_api.csv", index=False)

In [12]:
def add_one_hot(df, column):
    _df = df.copy()
    ohe = (pd.get_dummies(_df[column].apply(pd.Series).stack(), dtype="int")
             .groupby(level=0)
             .sum()
          )
    return pd.merge(_df, ohe, left_index=True, right_index=True) 

In [13]:
ohe_df = add_one_hot(df, column="terms")

In [14]:
ohe_df.head()

,patient_id,text,terms,Clinically healthy,Dermatologic disease,Gastrointestinal disease,Hematologic disease,Neurologic disease,Nonspecific,Nutritional disease,Ocular disease,Physical injury,Respiratory disease,Urogenital disease
0,1960540,unknown. A9 - orphaned. Feathers / Fur / Skin:...,[Clinically healthy],1,0,0,0,0,0,0,0,0,0,0
1,548645,HBC. HBC/NSF. Eyes / Ears / Mouth / Nares: squ...,[Physical injury],0,0,0,0,0,0,0,0,1,0,0
2,1905573,"Dragging leg, circling. Shock, Bleeding from M...",[Physical injury],0,0,0,0,0,0,0,0,1,0,0
3,2741655,Weak. NSF. Eyes / Ears / Mouth / Nares: Eyes s...,[Clinically healthy],1,0,0,0,0,0,0,0,0,0,0
4,2637768,Removed / Displaced (Juvenile) - Nest / Nest s...,[Nonspecific],0,0,0,0,0,1,0,0,0,0,0


In [15]:
ohe_df.to_csv(interim_data_dir/"wndp-api-ohe.csv", index=False)